In [1]:
!pip install gensim==3.6.0

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install zeugma==0.48
!pip install deslib

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


!pip install tensorflow==2.1.0

!pip install nltk 

In [4]:
from zeugma.embeddings import EmbeddingTransformer

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [6]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

In [8]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from deslib.util.aggregation import *

In [9]:
w2v = EmbeddingTransformer('word2vec')

In [10]:
glove = EmbeddingTransformer('glove')

In [11]:
fasttext = EmbeddingTransformer('fasttext')

In [47]:
fold = "F5"

In [48]:
uri_train = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_train_union.csv'.format(fold)
uri_val   = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_train_union.csv'.format(fold)
uri_teste = 'https://raw.githubusercontent.com/wvs2/data-hate/master/Folds/{}_test_union.csv'.format(fold)

df_train = pd.read_table(uri_train, sep=',')
df_val   = pd.read_table(uri_val, sep=',')
df_test = pd.read_table(uri_teste, sep=',')

In [49]:
train = df_train['text'].fillna(' ').apply(pre_processing)
val = df_val['text'].fillna(' ').apply(pre_processing)
test, class_test = df_test['text'].fillna(' ').apply(pre_processing), df_test['class']

In [50]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
cv.fit_transform(train.values.astype('U'))
     
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')
tfidf.fit_transform(train.values.astype('U'))

<24105x20340 sparse matrix of type '<class 'numpy.float64'>'
	with 164696 stored elements in Compressed Sparse Row format>

In [51]:
svm = {
    'CV': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=0.5, probability=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': fasttext,
    }
}

In [52]:
svm_clfs = {

}
for ext, clf in svm.items():
    svm_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [53]:
df_pred_train = pd.DataFrame(df_train['class'])
df_pred_val = pd.DataFrame(df_val['class'])
df_pred_test = pd.DataFrame(df_test['class'])

df_prob_val = pd.DataFrame(df_val['class'])
df_prob_train = pd.DataFrame(df_train['class'])
df_prob_test = pd.DataFrame(df_test['class'])
for ext, clf in svm_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["SVM-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["SVM-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["SVM-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "SVM-{}-{}".format(ext,clf.classes_[0]), 
    "SVM-{}-{}".format(ext,clf.classes_[1]), 
    "SVM-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [54]:
lr = {
    'CV': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': fasttext,
    }
}

In [55]:
lr_clfs = {

}
for ext, clf in lr.items():
    lr_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [56]:
for ext, clf in lr_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["LR-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["LR-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["LR-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "LR-{}-{}".format(ext,clf.classes_[0]), 
    "LR-{}-{}".format(ext,clf.classes_[1]), 
    "LR-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [57]:
rf = {
    'CV': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [58]:
rf_clfs = {

}
for ext, clf in rf.items():
    rf_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.3s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.6s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.9s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.2s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.4s remaining:    0.0s
b

[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    6.7s remaining:    0.0s
building tree 30 of 50
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    6.9s remaining:    0.0s
building tree 31 of 50
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:    7.1s remaining:    0.0s
building tree 32 of 50
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    7.4s remaining:    0.0s
building tree 33 of 50
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    7.6s remaining:    0.0s
building tree 34 of 50
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:    7.8s remaining:    0.0s
building tree 35 of 50
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    8.1s remaining:    0.0s
building tree 36 of 50
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    8.3s remaining:    0.0s
building tree 37 of 50
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    8.5s remaining:    0.0s
building tree 38 of 50
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:    8.7s remaining:  

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.7s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.8s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.9s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.1s remaining:    0.0s
building tree 10 of 50
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.2s remaining:    0.0s
building tree 11 of 50
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    1.3s remaining:    0.0s
building tree 12 of 50
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.4s remaining:    0.0s
building tree 13 of 50
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    1.6s remaining:    0.0s
building tree 14 of 50
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    1.7s remaining:    0.0s
building tree 15 of 50
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.8s remaining:    0

[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   14.6s remaining:    0.0s
building tree 36 of 50
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   15.0s remaining:    0.0s
building tree 37 of 50
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:   15.4s remaining:    0.0s
building tree 38 of 50
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:   15.8s remaining:    0.0s
building tree 39 of 50
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   16.2s remaining:    0.0s
building tree 40 of 50
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   16.6s remaining:    0.0s
building tree 41 of 50
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:   17.0s remaining:    0.0s
building tree 42 of 50
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:   17.4s remaining:    0.0s
building tree 43 of 50
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:   17.9s remaining:    0.0s
building tree 44 of 50
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:   18.3s remaining:  

In [59]:
for ext, clf in rf_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["RF-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["RF-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["RF-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "RF-{}-{}".format(ext,clf.classes_[0]), 
    "RF-{}-{}".format(ext,clf.classes_[1]), 
    "RF-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  2

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  2

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  2

[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  2

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [60]:
nb = {
    'CV': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': BernoulliNB(alpha=0.5, fit_prior=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT': fasttext,
    }
}

In [61]:
nb_clfs = {

}
for ext, clf in nb.items():
    nb_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [62]:
for ext, clf in nb_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["NB-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["NB-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["NB-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "NB-{}-{}".format(ext,clf.classes_[0]), 
    "NB-{}-{}".format(ext,clf.classes_[1]), 
    "NB-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [63]:
mlp = {
    'CV': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='lbfgs'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': fasttext,
    }
}

In [64]:
mlp_clfs = {

}
for ext, clf in mlp.items():
    mlp_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Iteration 1, loss = 0.44102993
Iteration 2, loss = 0.17873975
Iteration 3, loss = 0.11023509
Iteration 4, loss = 0.07944828
Iteration 5, loss = 0.06288832
Iteration 6, loss = 0.05244158
Iteration 7, loss = 0.04556884
Iteration 8, loss = 0.04190427
Iteration 9, loss = 0.03839187
Iteration 10, loss = 0.03616904
Iteration 11, loss = 0.03361984
Iteration 12, loss = 0.03184258
Iteration 13, loss = 0.03045529
Iteration 14, loss = 0.03008990
Iteration 15, loss = 0.02874713
Iteration 16, loss = 0.02766297
Iteration 17, loss = 0.02659370
Iteration 18, loss = 0.02776629
Iteration 19, loss = 0.02603360
Iteration 20, loss = 0.02537939


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.54862076
Iteration 2, loss = 0.47793878
Iteration 3, loss = 0.46109345
Iteration 4, loss = 0.44579565
Iteration 5, loss = 0.43485011
Iteration 6, loss = 0.42432524
Iteration 7, loss = 0.41313305
Iteration 8, loss = 0.40286059
Iteration 9, loss = 0.39259600
Iteration 10, loss = 0.38242310
Iteration 11, loss = 0.37143863
Iteration 12, loss = 0.36216707
Iteration 13, loss = 0.35130247
Iteration 14, loss = 0.34129064
Iteration 15, loss = 0.33235727
Iteration 16, loss = 0.32297033
Iteration 17, loss = 0.31342324
Iteration 18, loss = 0.30473830
Iteration 19, loss = 0.29648139
Iteration 20, loss = 0.28753231


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.62943493
Iteration 2, loss = 0.58936824
Iteration 3, loss = 0.58013646
Iteration 4, loss = 0.57224536
Iteration 5, loss = 0.56728296
Iteration 6, loss = 0.56257960
Iteration 7, loss = 0.55880889
Iteration 8, loss = 0.55594438
Iteration 9, loss = 0.55301241
Iteration 10, loss = 0.55048054
Iteration 11, loss = 0.54869568
Iteration 12, loss = 0.54580025
Iteration 13, loss = 0.54403689
Iteration 14, loss = 0.54263933
Iteration 15, loss = 0.54074081
Iteration 16, loss = 0.53980240
Iteration 17, loss = 0.53851851
Iteration 18, loss = 0.53668016
Iteration 19, loss = 0.53588525
Iteration 20, loss = 0.53410008


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.58443857
Iteration 2, loss = 0.47369914
Iteration 3, loss = 0.45342196
Iteration 4, loss = 0.44041670
Iteration 5, loss = 0.43127028
Iteration 6, loss = 0.42455092
Iteration 7, loss = 0.41637814
Iteration 8, loss = 0.41019982
Iteration 9, loss = 0.40556933
Iteration 10, loss = 0.39890945
Iteration 11, loss = 0.39468876
Iteration 12, loss = 0.38857403
Iteration 13, loss = 0.38349211
Iteration 14, loss = 0.37900878
Iteration 15, loss = 0.37293167
Iteration 16, loss = 0.36931833
Iteration 17, loss = 0.36423072
Iteration 18, loss = 0.35927594
Iteration 19, loss = 0.35532036
Iteration 20, loss = 0.34987266


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [65]:
for ext, clf in mlp_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["MLP-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["MLP-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["MLP-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "MLP-{}-{}".format(ext,clf.classes_[0]), 
    "MLP-{}-{}".format(ext,clf.classes_[1]), 
    "MLP-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [66]:
extra = {
    'CV': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [67]:
extra_clfs = {

}
for ext, clf in extra.items():
    extra_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.3s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.7s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    3.2s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.6s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    4.0s remaining:    0.0s
b

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.7s remaining:    0.0s
building tree 10 of 50
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.8s remaining:    0.0s
building tree 11 of 50
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.9s remaining:    0.0s
building tree 12 of 50
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.0s remaining:    0.0s
building tree 13 of 50
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    1.0s remaining:    0.0s
building tree 14 of 50
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    1.1s remaining:    0.0s
building tree 15 of 50
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.2s remaining:    0.0s
building tree 16 of 50
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    1.3s remaining:    0.0s
building tree 17 of 50
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    1.3s remaining:    0.0s
building tree 18 of 50
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.4s remaining:  

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.1s remaining:    0.0s
building tree 41 of 50
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:    1.2s remaining:    0.0s
building tree 42 of 50
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:    1.2s remaining:    0.0s
building tree 43 of 50
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:    1.2s remaining:    0.0s
building tree 44 of 50
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:    1.2s remaining:    0.0s
building tree 45 of 50
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.3s remaining:    0.0s
building tree 46 of 50
[Parallel(n_jobs=1)]: Done  46 out of  46 | elapsed:    1.3s remaining:    0.0s
building tree 47 of 50
[Parallel(n_jobs=1)]: Done  47 out of  47 | elapsed:    1.3s remaining:    0.0s
building tree 48 of 50
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    1.4s remaining:    0.0s
building tree 49 of 50
[Parallel(n_jobs=1)]: Done  49 out of  49 | elapsed:    1.4s remaining:  

In [68]:
for ext, clf in extra_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["EXTRA-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["EXTRA-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["EXTRA-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "EXTRA-{}-{}".format(ext,clf.classes_[0]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[1]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  2

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  3

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elaps

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elaps

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [69]:
knn = {
    'CV': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': fasttext,
    }
}

In [70]:
knn_clfs = { }
for ext, clf in knn.items():
    knn_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [71]:
for ext, clf in knn_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["KNN-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["KNN-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["KNN-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "KNN-{}-{}".format(ext,clf.classes_[0]), 
    "KNN-{}-{}".format(ext,clf.classes_[1]), 
    "KNN-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


# CNN

In [72]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=300

y_train = to_categorical(df_train['class'])
y_val = to_categorical(df_val['class'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train)

seq_train = tokenizer.texts_to_sequences(train)
seq_val = tokenizer.texts_to_sequences(val)
seq_test = tokenizer.texts_to_sequences(test)

data_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
data_val = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [73]:
cnn_cv = get_CNN(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid')
cnn_tfidf = get_CNN(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='softmax')
cnn_w2v = get_CNN(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)
cnn_glove = get_CNN(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True)
cnn_fast = get_CNN(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 300)          6000300   
_________________________________________________________________
dropout_10 (Dropout)         (None, 300, 300)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_10 (Activation)   (None, 256)              

In [74]:
cnn_cv.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=20)

Epoch 1/20
1206/1206 [==============================] - 27s 22ms/step - loss: 0.6498 - accuracy: 0.7069 - val_loss: 0.5465 - val_accuracy: 0.7429
Epoch 2/20
1206/1206 [==============================] - 26s 22ms/step - loss: 0.4914 - accuracy: 0.7804 - val_loss: 0.4145 - val_accuracy: 0.8423
Epoch 3/20
1206/1206 [==============================] - 26s 22ms/step - loss: 0.4022 - accuracy: 0.8410 - val_loss: 0.3687 - val_accuracy: 0.8522
Epoch 4/20
1206/1206 [==============================] - 26s 22ms/step - loss: 0.3624 - accuracy: 0.8589 - val_loss: 0.3306 - val_accuracy: 0.8822
Epoch 5/20
1206/1206 [==============================] - 26s 22ms/step - loss: 0.3354 - accuracy: 0.8732 - val_loss: 0.2969 - val_accuracy: 0.8965
Epoch 6/20
1206/1206 [==============================] - 27s 22ms/step - loss: 0.3135 - accuracy: 0.8845 - val_loss: 0.2846 - val_accuracy: 0.8949
Epoch 7/20
1206/1206 [==============================] - 26s 22ms/step - loss: 0.2997 - accuracy: 0.8927 - val_loss: 0.2785 -

In [75]:
cnn_tfidf.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

Epoch 1/20
121/121 [==============================] - 26s 213ms/step - loss: 0.8158 - accuracy: 0.6359 - val_loss: 0.6652 - val_accuracy: 0.6358
Epoch 2/20
121/121 [==============================] - 26s 214ms/step - loss: 0.6117 - accuracy: 0.7239 - val_loss: 0.5675 - val_accuracy: 0.7555
Epoch 3/20
121/121 [==============================] - 26s 213ms/step - loss: 0.5573 - accuracy: 0.7539 - val_loss: 0.5321 - val_accuracy: 0.7649
Epoch 4/20
121/121 [==============================] - 26s 214ms/step - loss: 0.5263 - accuracy: 0.7642 - val_loss: 0.5002 - val_accuracy: 0.7726
Epoch 5/20
121/121 [==============================] - 26s 214ms/step - loss: 0.4945 - accuracy: 0.7744 - val_loss: 0.4678 - val_accuracy: 0.7795
Epoch 6/20
121/121 [==============================] - 26s 214ms/step - loss: 0.4638 - accuracy: 0.7886 - val_loss: 0.4372 - val_accuracy: 0.8197
Epoch 7/20
121/121 [==============================] - 26s 212ms/step - loss: 0.4412 - accuracy: 0.8085 - val_loss: 0.4120 - val_ac

121/121 [==============================] - 3s 28ms/step - loss: 0.3421 - accuracy: 0.8729 - val_loss: 0.2857 - val_accuracy: 0.8926
Epoch 18/20
121/121 [==============================] - 3s 28ms/step - loss: 0.3402 - accuracy: 0.8730 - val_loss: 0.2715 - val_accuracy: 0.8993
Epoch 19/20
121/121 [==============================] - 3s 28ms/step - loss: 0.3376 - accuracy: 0.8753 - val_loss: 0.2722 - val_accuracy: 0.8986
Epoch 20/20
121/121 [==============================] - 3s 28ms/step - loss: 0.3331 - accuracy: 0.8767 - val_loss: 0.2659 - val_accuracy: 0.9002
Epoch 1/20
121/121 [==============================] - 26s 212ms/step - loss: 0.5296 - accuracy: 0.7859 - val_loss: 0.3490 - val_accuracy: 0.8662
Epoch 2/20
121/121 [==============================] - 25s 210ms/step - loss: 0.3492 - accuracy: 0.8678 - val_loss: 0.2951 - val_accuracy: 0.8931
Epoch 3/20
121/121 [==============================] - 26s 212ms/step - loss: 0.3102 - accuracy: 0.8850 - val_loss: 0.2968 - val_accuracy: 0.8940
E

In [76]:
# print(df_prob_train['CNN-CV-0'])
# # for ext in ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']:
# #   for i in range(0,3):
# #     df_prob_train = df_prob_train.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_val = df_prob_val.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_test = df_prob_test.drop(columns=['CNN-{}-{}'.format(ext, i)])

In [77]:
print(df_prob_train.columns)

Index(['class', 'SVM-CV-0', 'SVM-CV-1', 'SVM-CV-2', 'SVM-TFIDF-0',
       'SVM-TFIDF-1', 'SVM-TFIDF-2', 'SVM-W2V-0', 'SVM-W2V-1', 'SVM-W2V-2',
       ...
       'KNN-TFIDF-2', 'KNN-W2V-0', 'KNN-W2V-1', 'KNN-W2V-2', 'KNN-GLOVE-0',
       'KNN-GLOVE-1', 'KNN-GLOVE-2', 'KNN-FAST-0', 'KNN-FAST-1', 'KNN-FAST-2'],
      dtype='object', length=106)


In [78]:
# df_pred_test = df_pred_test.drop(['CNN-CV'], axis=1)
# print(df_pred_test.columns)
# CV
cols = ["CNN-CV-0", "CNN-CV-1", "CNN-CV-2"]
df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# print(df_pred_test.columns)

# # Probabilidades
  
df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-TF-0", "CNN-TF-1", "CNN-TF-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-W2V-0", "CNN-W2V-1", "CNN-W2V-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-GLOVE-0", "CNN-GLOVE-1", "CNN-GLOVE-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-FAST-0", "CNN-FAST-1", "CNN-FAST-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [79]:
df_prob_train['class']

0        2
1        1
2        1
3        1
4        1
        ..
24100    0
24101    1
24102    2
24103    0
24104    0
Name: class, Length: 24105, dtype: int64

In [80]:
df_pred_train.to_csv("pred_train_union_{}.csv".format(fold))
df_pred_val.to_csv("pred_val_union_{}.csv".format(fold))
df_pred_test.to_csv("pred_test_union_{}.csv".format(fold))

In [81]:
df_prob_train.to_csv("prob_train_union_{}.csv".format(fold))
df_prob_val.to_csv("prob_val_union_{}.csv".format(fold))
df_prob_test.to_csv("prob_test_union{}.csv".format(fold))